# Ising-Lenz Ergodicity: Generate Distributional Power-laws in Gamma Distribution

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

This notebook generates power-laws in the form, $P(\Gamma_{G}(t)) \sim \Gamma_{G}(t)^{-\alpha}$. Using outputs from generate gamma. power-laws are fitted and results are repored with the bias corrected bootstrap confidence intervals. 

The resulting dataset is called `ising1DrateErgodicityPowerlawsDist.rds`. 

$\Omega_{G}(t)$ : Modified TM G-metric, ergodic convergence.  
$\Gamma_{G}(t)$ : Rate of ergodic convergence.  
$\Kappa_{G}(t)$ : Inverse rate of ergodic convergence.  


* Workhorse of the data generation is method `dist_power_law`. The RDS file is simply 
 a data frame for each parameters and their power-law fit information.
* Parameters are hardcoded in paralell method below.
     * Sizes at Metropolis (`transP=1`) and Glauber (`transP=2`) Dynamics `N=512, 1024, 1536`.
     * Temperature ranges `seq(from = 0.5, by = 0.06, length.out = 16)`.
     * `J=1.0 H=1.0`.
     * MC steps up-to `2e6`. 
       Each `by_ = 1000` point is used for fitting (This is important).


In [ ]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library("igraph")
library(parallel)       # Core package

source("../src/power_utilities.R");


In [ ]:
  library(boot)

  get_mean_boot95_bca <- function(data) {
    # Bias-corrected bootstraped means and 95$ CI.
    mean_boot <- function(data, indices) {
      return(mean(data[indices]))
    }
    boot_result <- boot(data = as.numeric(data), statistic = mean_boot, R = 1000)
    boot_means <- boot_result$t
    ci <- boot::boot.ci(boot_result, type = "bca")$bca[4:5]
    list("mean" = boot_result$t0, "lower" = ci[1], "upper" = ci[2], "ci" = ci)
  }

dist_power_law <- function(accept_time, gamma_, by_ = 1000) {
  # fit power-law given RunSim's result
  # P(Gamma) ~ Gamma^alpha
  ll <- length(gamma_)
  ix <- seq(1, ll, by = by_)
  accept_time <- accept_time[ix]
  gamma_ <- gamma_[ix]
  pf <- igraph::fit_power_law(gamma_)
  alpha <- pf$alpha
  ix <- which(gamma_ == pf$xmin)[1]
  tmin <- accept_time[ix]
  D <- pf$KS.stat
  pairlist(
    "alpha" = alpha,
    "tmin" = tmin,
    "D" = D
  )
}

dist_power_law_ci <- function(result, by_=1000) {
    nrepeat <- length(result$accept_times_repeats)
    alphas <- vector("numeric", nrepeat)
    optDs <- vector("numeric", nrepeat)
    opt_tmins <- vector("numeric", nrepeat)
    for(i in 1:nrepeat) {
        gamma_ <- result$gamma_repeats[[i]]
        accept_time <- result$accept_times_repeats[[i]]
        ixx <- which(accept_time < 2e6)
        pf <- dist_power_law(accept_time[ixx], gamma_[ixx], by_)
        alphas[i] <- pf$alpha
        optDs[i] <- pf$D
        opt_tmins[i] <- pf$tmin
    }
    # alpha
    alpha_s <- get_mean_boot95_bca(as.numeric(alphas))
    alpha_mean <- alpha_s$mean
    alpha_boot95ci <- c(alpha_s$lower, alpha_s$upper)
    # Ds
    D_s <- get_mean_boot95_bca(as.numeric(optDs))
    D_mean <- D_s$mean
    D_boot95ci <- c(D_s$lower, D_s$upper)
    # tmins
    tmin_s <- get_mean_boot95_bca(as.numeric(opt_tmins))
    tmin_mean <- tmin_s$mean
    tmin_boot95ci <- c(tmin_s$lower, tmin_s$upper)
    pl_info = list(
        "N"=result$N,
        "ikBT"=result$ikBT,
        "dynamics"=result$dynamics,
        "H"=result$H,
        "J"=result$J,
        "alpha_mean"=alpha_mean,
        "alpha_upper"=alpha_s$upper,
        "alpha_lower"=alpha_s$lower,
        "D_mean"=D_mean,
        "D_upper"=D_s$upper,
        "D_lower"=D_s$lower,
        "tmin_mean"=as.integer(tmin_mean),
        "tmin_upper"=as.integer(tmin_s$upper),
        "tmin_lower"=as.integer(tmin_s$lower)
    )
    pl_info
}

In [ ]:
ergoRates <- readRDS("../datasets/ising1DrateErgodicity.rds")

In [ ]:
nresults <- length(ergoRates)
for(j in 1:nresults){
    result <- ergoRates[[j]]
    to_null <- function(x) { 
        if (is.null(x)) NA else x 
    }
    df_pl <- as.data.frame(lapply(dist_power_law_ci(result), to_null))
    if(j == 1) {
        df_pl_gamma = df_pl
    } else {
        df_pl_gamma <- rbind.data.frame(df_pl_gamma, df_pl)
    }
}

In [ ]:
saveRDS(df_pl_gamma, "../datasets/ising1DrateErgodicityPowerlawsDistribution.rds")